# Construindo o vocabulário NLP

Esse texto abordará o conceito de vocabulário e os conceitos necessários para sua construção, abordando os principais métodos da área de NLP para essa tarefa.

## Tokenização

O primeiro passo para a construção de um vocabulário é a quebra do texto em partes menores, chamadas **tokens**. **Tokenização** pode ser entedido como o processo de dividir o texto em fatias menores, mas que ainda contenham significado. Tokens geralmente são palavras, mas podem englobar números, pontuações, símbolos e emoticons. 

In [ ]:
texto = "O rato roeu a roupa do rei de roma"
texto.split()

['O', 'rato', 'roeu', 'a', 'roupa', 'do', 'rei', 'de', 'roma']

### Problemas com a Tokenização

O processo de tokenização mais básico considera que os termos separados por espaços contém significado independentemente. Entretanto, isto não é sempre o caso. Vejamos as frases abaixo:

In [ ]:
texto = "A cidade do Rio de Janeiro é muito bonita"
texto.split()

['A', 'cidade', 'do', 'Rio', 'de', 'Janeiro', 'é', 'muito', 'bonita']

Neste caso, *Rio de Janeiro* deveria ser um único token pois, no contexto, as palavras *Rio*, *de* e *Janeiro* não apresentam significado individual.

Tokens podem ter tamanhos variados. O mais comum é o **unigrama**, que contém apenas uma palavra, mas podem haver **n-gramas** de qualquer tamanho.

A ideia é criar tokenizers que possam lidar de forma adequada com o contexto do qual o corpus textual foi extraído.

### Diferentes tipos de Tokenizadores

**Tokenizers baseados em Expressões Regulares**

Expressões regulares são conjuntos de caracteres que determinam um padrão textual específico. São a forma mais popular de busca de padrões em textos. 

Por exemplo, se estamos em busca de um CPF em uma frase, sabemos que ele obedece a forma XXX.XXX.XXX-XX onde cada X é necessáriamente um número entre 0 e 9. Essa forma padrão do CPF é dita uma expresão regular, e com ela somos capazes de identificar qualquer CPF dentro de um texto. O python nativamente já possui uma biblioteca para trabalhar com expressões regulares.

Saiba mais no [link](https://www.w3schools.com/python/python_regex.asp). 

In [ ]:
# Exemplo de busca por CPF
import re

expressao_regular = "[0-9][0-9][0-9].[0-9][0-9][0-9].[0-9][0-9][0-9]-[0-9][0-9]"

frase1 = "Meu CPF é 123.456.789-11"
frase2 = "O CPF 987.654.321-10 pertence a Fulano"

print( re.findall(expressao_regular, frase1) )
print( re.findall(expressao_regular, frase2) )

['123.456.789-11']
['987.654.321-10']


Com o uso de Regex (Regualar Expressions), somos capazes de determinar padrões específicos no texto que devem ser captuados pelo tokenizador.

O NLTK provê uma funcionalidade **regular expressions-based
tokenizers** (RegexpTokenizer), que faz exatamente isso.

In [ ]:
from nltk.tokenize import RegexpTokenizer

# O símbolo | representa o conectivo lógico OU
expressao_regular = '[0-9]{3}.[0-9]{3}.[0-9]{3}-[0-9]{2}|Rio de Janeiro|São Paulo|\w+|\S+'

frase1 = "Fulano mora no Rio de Janeiro, seu CPF é 987.654.321-10"
frase2 = "Cicrano, de cpf 123.456.789-11, mora em São Paulo"

tokenizer = RegexpTokenizer(expressao_regular)
print(tokenizer.tokenize(frase1))
print(tokenizer.tokenize(frase2))

['Fulano', 'mora', 'no', 'Rio de Janeiro', ',', 'seu', 'CPF', 'é', '987.654.321-10']
['Cicrano', ',', 'de', 'cpf', '123.456.789-11', ',', 'mora', 'em', 'São Paulo']


**TweetTokenizer**

Um exemplo de tokenizer contextual é o TweetTokenizer, feito especificamente para lidar com a linguagem utilizada em Tweets.

In [ ]:
from nltk.tokenize import TweetTokenizer

tweet1 = "@fulano_de_tal Consegui comprar o meu carro própriooooooo!!! :) :-D #felicidade #carroproprio"
tokenizer = TweetTokenizer()
tokenizer.tokenize(tweet1)

['@fulano_de_tal',
 'Consegui',
 'comprar',
 'o',
 'meu',
 'carro',
 'própriooooooo',
 '!',
 '!',
 '!',
 ':)',
 ':-D',
 '#felicidade',
 '#carroproprio']

O tokenizer foi capaz de lidar com as hastags *#felicidade* e *#carroproprio* e os emoticons *:-D* e *:)*.

Além disso, ele possui dois parâmetros opcionais. O *reduce_len* serve para reduzir o uso excessivo de caracteres ao fim de uma palavra e o *strip_handles*, para remover cabeçalhos relacionados ao twitter.


In [ ]:
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
tokenizer.tokenize(tweet1)

['Consegui',
 'comprar',
 'o',
 'meu',
 'carro',
 'própriooo',
 '!',
 '!',
 '!',
 ':)',
 ':-D',
 '#felicidade',
 '#carroproprio']

## Entendendo normalização de palavras

Na maioria dos casos, não precisamos conter cada palavra única do texto em nosso vocabulário. Textos são um tipo de informação muito ruidosa, preparar e selecionar corretamente quais palavras aparecerão no vocabulário ajuda a aumentar a representatividade da informação textual. As palavras *sou*, *é* e *somos* podem ser reduzidas a sua forma infinitiva *ser* sem grande perda de significado. De forma análoga, desinências verbais também podem ser removidas para manutenção do número de termos.

Palavras como *a*, *uma* e *o*, que ocorrem frequêntemente, geralmente não são decisivas e não carregam muita informação para tarefas de Machine Learning, podendo ser totalmente removidas. Esse processo se chama **remoção de stopwords**.

Vale salientar que essa etapa do processamento depende bastante do contexto trabalhado. 

### Stemming

Stemming (Stemização) é o processo de remover alguns caracteres de  palavras inflexionadas, na tentativa de reduzi-las ao seu **radical** (stem).

Por exemplo, as palavras *sapato*, *sapateiro* e *sapataria* são formadas pelo o radical *sapat* e por um **afixo** responsável pela inflexão da palavra. Ao reduzir todos os termos a *sapat*, tentamos enxugar nosso vocabulário enquanto mantemos o significado fundamental de cada um deles.

O **Snowball stemmer** é um algoritmo de stemização disponível na NLTK que suporta a língua portuguesa.



In [ ]:
from nltk.stem.snowball import SnowballStemmer

print(SnowballStemmer.languages)

('arabic', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'porter', 'portuguese', 'romanian', 'russian', 'spanish', 'swedish')


Vamos aplicar o stemmer em alguns exemplos.

In [ ]:
stemmer = SnowballStemmer(language='portuguese')

palavras = ["casa", "casinha", "casebre", 
            "porta", "portaria", "portão", 
            "amar", "amaria", "amarei",
            "corri", "correis","correríamos"]

for palavra in palavras:
  print( "{:12s} - {:s}".format(palavra, stemmer.stem(palavra)) )

casa         - cas
casinha      - casinh
casebre      - casebr
porta        - port
portaria     - port
portão       - portã
amar         - amar
amaria       - amar
amarei       - amar
corri        - corr
correis      - corr
correríamos  - corr


### Lemmatization

Enquanto o processo de Stemming envolve somente a remoção de caracteres ao fim da palavra, o Lemmatization tenta reduzir uma palavra a sua forma base considerando o seu contexto. Isso ajuda especialmente a agrupar palavras semelhantes, que serão reduzidas a uma mesma forma fundamental.

Palavras como *leiteira*, *leiteiro* e *leitoso*, serão reduzidas a sua forma base *leite*. Verbos conjugados serão trazidos a sua forma infinitiva.

Algoritmos de Lemmatization levam em conta o contexto da palavra, sua Classe gramatical, referida normalmente como **Part-of-speech (POS) tag**, etc.

A NLTK não possui nenhum lemmatizer que suporte o português.

Prosseguiremos utilizando a biblioteca Stanza, baseado-nos no link abaixo.


https://lars76.github.io/2018/05/08/portuguese-lemmatizers.html#3

In [ ]:
!pip install stanza

     |████████████████████████████████| 235kB 7.0MB/s 


In [ ]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt')

2021-01-03 06:21:14 INFO: Downloading default packages for language: pt (Portuguese)...
2021-01-03 06:21:29 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-01-03 06:21:29 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2021-01-03 06:21:30 INFO: Use device: cpu
2021-01-03 06:21:30 INFO: Loading: tokenize
2021-01-03 06:21:30 INFO: Loading: mwt
2021-01-03 06:21:30 INFO: Loading: pos
2021-01-03 06:21:31 INFO: Loading: lemma
2021-01-03 06:21:31 INFO: Loading: depparse
2021-01-03 06:21:32 INFO: Done loading processors!


In [ ]:
texto = "Éramos dois e contrários. Ela encobrindo com a palavra o que eu publicava pelo silêncio"
texto_info = nlp(texto)

In [ ]:
print("{:12s}   {:12s}   {:s}\n".format("Palavra", "POS tag", "Lemma") )
for sentenca in texto_info.sentences:
  for palavra in sentenca.words:
    print("{:12s}   {:12s}   {:s}".format(palavra.text, palavra.upos, palavra.lemma) )
  print(40*"-")

Palavra        POS tag        Lemma

Éramos         AUX            ir
dois           NUM            dois
e              CCONJ          e
contrários     ADJ            contrário
.              PUNCT          .
----------------------------------------
Ela            PRON           ela
encobrindo     VERB           encobrir
com            ADP            com
a              DET            o
palavra        NOUN           palavra
o              PRON           o
que            PRON           que
eu             PRON           eu
publicava      VERB           publicar
por            ADP            por
o              DET            o
silêncio       NOUN           silêncio
----------------------------------------


Como podemos ver, o lematizador tenta encontrar a forma base de cada palavra. 

Assim como o Stemmer, ele não é perfeito e comete alguns erros.
Por exemplo, o lemma de *Éramos* deveria ser *Ser*.

Entretanto, quando lidando com o termo *pelo*, o lemmatizer consegue um sucesso que o stemmer seria incapaz, já que a palavra *pelo* é uma aglutinação do verbo *por* mais o artigo *o*, comportamento indetectável pelo stemmer.

Outra informação que podemos ver é a classe gramatical detectada. As siglas seguem o padrão abaixo:

* ADJ: adjetivo
* ADP: adposição (sempre preposições em portug)
* ADV: advérbio
* AUX: auxiliar
* CCONJ: conjunção coordenada
* DET: determinante
* INTJ: interjeição
* NOUN: substantivo
* NUM: numeral
* PART: partícula
* PRON: pronome
* PROPN: nome próprio
* PUNCT: pontuação
* SCONJ: conjunção subordinada
* SYM: símbolo
* VERB: verbo
* X: outro

### Remoção de Stopwords

O que são stopwords?

Stopwords são palavras frequentes em um corpus textual que não carregam muito significado na maioria dos contextos. Geralmente são palavras necessárias para gerar coesão gramatical e coerência das frases.

Não existe um consenso universal de quais palavras são definitivamente stopwords, vai depender da aplicação. Algumas bibliotecas de Machine Learning e NLP possuem listas de stopwords disponíveis em alguns idiomas, mas elas devem ser modificadas caso necessário.

Vamos dar uma olhada na lista do NLTK de stopwords em português.

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
portugues_stopwords = stopwords.words('portuguese')
", ".join(portugues_stopwords)

'de, a, o, que, e, é, do, da, em, um, para, com, não, uma, os, no, se, na, por, mais, as, dos, como, mas, ao, ele, das, à, seu, sua, ou, quando, muito, nos, já, eu, também, só, pelo, pela, até, isso, ela, entre, depois, sem, mesmo, aos, seus, quem, nas, me, esse, eles, você, essa, num, nem, suas, meu, às, minha, numa, pelos, elas, qual, nós, lhe, deles, essas, esses, pelas, este, dele, tu, te, vocês, vos, lhes, meus, minhas, teu, tua, teus, tuas, nosso, nossa, nossos, nossas, dela, delas, esta, estes, estas, aquele, aquela, aqueles, aquelas, isto, aquilo, estou, está, estamos, estão, estive, esteve, estivemos, estiveram, estava, estávamos, estavam, estivera, estivéramos, esteja, estejamos, estejam, estivesse, estivéssemos, estivessem, estiver, estivermos, estiverem, hei, há, havemos, hão, houve, houvemos, houveram, houvera, houvéramos, haja, hajamos, hajam, houvesse, houvéssemos, houvessem, houver, houvermos, houverem, houverei, houverá, houveremos, houverão, houveria, houveríamos, hou

A lista acima contém uma coleção genérica do que o NLTK entende por Stopwords em português. Entretanto, como já discutido acima, em contextos específicos algumas dessas palavras podem ser necessárias.

O procedimento de remoção de stopwords geralmente ocorre logo após a tokenização.

In [ ]:
frase = "Você já reparou nos olhos dela? São assim de cigana oblíqua e dissimulada"
tokens = frase.split()
frase_sem_stopwords = [ token for token in tokens if token not in portugues_stopwords ]
frase_sem_stopwords

['Você',
 'reparou',
 'olhos',
 'dela?',
 'São',
 'assim',
 'cigana',
 'oblíqua',
 'dissimulada']

### Case folding

Outra estratégia adotada para normalizar os textos é o **case folding**. Em linhas gerais, o procedimento vai deixar todas as letras em **lowercase**.

Neste caso, as palavras *Casa* e *casa* serão resumidas somente ao termo *casa*, o que não trás muita diferença de significado.

Entretanto, em casos onde a presença de nomes próprios e siglas for de maior importância, o case folding pode não ser uma boa opção. Por exemplo, o termo *Rio*, referente à cidade, perde totalmente o significado se for reduzido à forma *rio*.

O ideal é que possamos detectar quais palavras podem ou não sofrer a alteração de caso.

A maioria das bibliotecas de NLP já trabalha automaticamente com os termos em lowercase. Abaixo podemos ver uma implementação simples em python nativo. 

In [ ]:
frase = "Esta É Uma Frase"
frase.lower()

'esta é uma frase'

### N-Grams

Até o momento, todas as nossas análises se basearam em tokens de tamanho 1, que apenas comtém uma palavra. Entretanto, sentenças geralmente contém termos compostos, como *Rio de Janeiro*, *São Paulo*, *fim de semana* e *sala de estar*. 

Esses termos carregam significados inerentes de sua forma composta, e não podem ser separados. Por mais que termos compostos sejam relativamente incomuns, eles carregam uma boa quantidade de informação, e técnicas devem ser aplicadas para que eles possam ser detectados.

No geral, esses termos são agrupados em n-grams. Um n-gram é um grupo de n palavras sequênciais em um texto. Dessa forma, quando n é 1, temos a tokenização normal. Quando n é 2, temos a tokenização com bigramas, que é um caso bem comum nas aplicações de NLP.

A maioria dos casos se resume a trigramas ou menos. Em geral, algoritmos de NLP utilizam-se simultaneamente de unigramas, bigramas e trigramas.

Os códigos abaixo mostram como gerar n-gramas com o NLTK. 


In [ ]:
from nltk.util import ngrams
frase = "O cantor Roberto Carlos nasceu no dia 19 de abril em Cachoeiro de Itapemirim"

tokens = frase.split()

In [ ]:
bigramas = list(ngrams(tokens, 2))
[" ".join(token) for token in bigramas]

['O cantor',
 'cantor Roberto',
 'Roberto Carlos',
 'Carlos nasceu',
 'nasceu no',
 'no dia',
 'dia 19',
 '19 de',
 'de abril',
 'abril em',
 'em Cachoeiro',
 'Cachoeiro de',
 'de Itapemirim']

In [ ]:
trigramas = list(ngrams(tokens, 3))
[" ".join(token) for token in trigramas]

['O cantor Roberto',
 'cantor Roberto Carlos',
 'Roberto Carlos nasceu',
 'Carlos nasceu no',
 'nasceu no dia',
 'no dia 19',
 'dia 19 de',
 '19 de abril',
 'de abril em',
 'abril em Cachoeiro',
 'em Cachoeiro de',
 'Cachoeiro de Itapemirim']

Como podemos ver acima, os termos *Roberto Carlos* e *Cachoeiro de Itapemirim* só puderam ser verdadeiramente capturados pelo uso dos n-gramas.

## Resumo

Esse material abordou uma série de etapas decisivas na construção de um vocabulário. O pré processamento dos dados é uma etapa crucial nas aplicações de Machine Learning, e ganha uma contexto próprio quando aplicado ao processamento de sequências textuais.

Os métodos ensinados acima são peças para que montemos uma pipiline de pré processamento, e podem ser mais ou menos necessários em cada caso de uso. O fato é, quando usados adequadamente, irão ajudar os passos subsequêntes do modelo de Machine Learning, consequêntemente gerando resultados melhores.